# Histopathologic Cancer Detection

The goal of this project is to create an algorithm to identify metastatic cancer in small image patches taken from larger digital pathology scans.

This project was built as a submission to this Kaggle competition: https://www.kaggle.com/c/histopathologic-cancer-detection/overview